In [1]:
from __future__ import division
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np

In [2]:
import os
from PIL import Image
import numpy
#from PIL import Image


import pathlib

In [3]:
def open_folder():
    paths = []
    for root, dirs, files in os.walk("./gen_midi"):
        for f in files:
            path = os.path.join(root,f)
            path = os.path.abspath(path)
            paths.append(path)

    return paths

In [4]:
def preprocess_image(image):
    image = tf.image.decode_png(image, channels=3)
    image = image / np.array(255, dtype=np.float32)
    return image

def load_and_preprocess_image(path, label):
    image = tf.read_file(path)
    return (preprocess_image(image), label)

In [5]:
paths = open_folder()

In [6]:
tf.enable_eager_execution()

In [7]:
label_num = {'Electronic': 0, 'Jazz': 1, 'Rock': 2}


In [8]:
labels = [x.split("/")[7] for x in paths]
#print(labels)
#print(label_num[labels[0]])
labels = [label_num[x] for x in labels]

In [9]:
path_ds = tf.data.Dataset.from_tensor_slices(paths)
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(labels, tf.int64))


dataset = tf.data.Dataset.from_tensor_slices((paths, labels))

In [10]:
print(dataset)

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int32)>


In [11]:
DATASET_SIZE = len(paths)


ds = dataset.shuffle(buffer_size=int(DATASET_SIZE))

In [12]:




print(ds)

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int32)>


In [13]:
ds = ds.map(load_and_preprocess_image)


In [14]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,8))
for n, (image, label) in enumerate(ds.take(4)):
    plt.imshow(image)
    plt.xlabel(label)
    plt.show()
    #print(label)

Instructions for updating:
Colocations handled automatically by placer.


<Figure size 800x800 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [15]:
AUTOTUNE = tf.data.experimental.AUTOTUNE


BATCH_SIZE = 32
DATASET_SIZE = len(paths)

# completely shuffled.
#ds = image_label_ds.shuffle(buffer_size=int(DATASET_SIZE))
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)
# `prefetch` lets the dataset fetch batches, in the background while the model is training.
ds = ds.prefetch(buffer_size=AUTOTUNE)

In [16]:
steps_per_epoch= int(tf.ceil(len(paths)/BATCH_SIZE).numpy())
#print(steps_per_epoch)

#DATASET_SIZE = len(paths)
train_size = int(0.7 * DATASET_SIZE)
test_size = int(0.3 * DATASET_SIZE)

train_dataset = ds.take(train_size)
test_dataset = ds.skip(train_size)


In [17]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(81, 80, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 79, 78, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 39, 39, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 37, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1048640   
__________

In [19]:
model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [20]:
print(ds)

<DatasetV1Adapter shapes: ((?, ?, ?, 3), (?,)), types: (tf.float32, tf.int32)>


In [21]:
model.fit(train_dataset, epochs=2, steps_per_epoch=steps_per_epoch)


Epoch 1/2
2550/2550 [==============================] - 1097s 430ms/step - loss: 0.2156 - acc: 0.9185
Epoch 2/2
2550/2550 [==============================] - 886s 347ms/step - loss: 0.0822 - acc: 0.9699


In [24]:
model.save('test4.h5')

RuntimeError: Unable to create link (name already exists)

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 79, 78, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 39, 39, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 37, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1048640   
__________

In [38]:
model.evaluate(test_dataset, steps=1)

1/1 [==============================] - 697s 697s/step - loss: 0.0332 - acc: 1.0000


[0.033216655254364014, 1.0]

In [39]:
model.predict(test_dataset, steps=1)

array([[1.54648973e-07, 2.11948397e-08, 9.99999881e-01],
       [3.87042522e-01, 1.04902253e-01, 5.08055210e-01],
       [1.81664333e-07, 9.96652186e-01, 3.34763736e-03],
       [4.07858133e-05, 9.98010337e-01, 1.94886432e-03],
       [2.03765160e-03, 4.14835988e-03, 9.93813992e-01],
       [5.11880160e-07, 9.97693598e-01, 2.30585458e-03],
       [3.27811364e-07, 4.08442169e-08, 9.99999642e-01],
       [2.49817572e-06, 4.94503456e-06, 9.99992609e-01],
       [2.86528096e-02, 6.43240847e-03, 9.64914739e-01],
       [1.91147532e-02, 1.86928943e-01, 7.93956280e-01],
       [6.63815854e-07, 7.50136678e-05, 9.99924302e-01],
       [1.01740625e-04, 7.12798783e-06, 9.99891162e-01],
       [9.54788327e-01, 7.03132973e-05, 4.51412797e-02],
       [1.84852059e-03, 1.81039228e-04, 9.97970402e-01],
       [9.83593404e-01, 4.99954467e-06, 1.64015032e-02],
       [9.96886432e-01, 1.42535574e-08, 3.11350427e-03],
       [1.92377047e-06, 7.32093142e-08, 9.99997973e-01],
       [2.42746309e-11, 3.01016